本代码旨在提升latex生成的效果

In [1]:
import json
datas=[]
with open('aime_amc_new.jsonl','r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        data=json.loads(line)
        # if data['split']=='test':
        datas.append(data)
datas2=[]
with open('1111latex.jsonl','r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        data=json.loads(line)
        # if data['split']=='test':
        datas2.append(data)

In [2]:
# 提取latex提示词
latex=[]
for i in range(len(datas2)):
    cut = len(datas[i]['formal_statement'])
    latex.append(datas2[i]['formal_statement'][cut:])

In [3]:
import re
# 根据编号进行分组
# 正则表达式匹配编号和随后的文本
def seg_text(text):
    pattern = r'(-- |\n)(\d+)\. '
    pattern2 = r'Step \d+'
    matches = re.finditer(pattern, text, re.DOTALL)
    matches2 = re.finditer(pattern2, text, re.DOTALL)
    segs=[]
    segs2=[]
    for match in matches:
        segs.append(match.start())
    for match in matches2:
        segs2.append(match.start())
    if len(segs2)> len(segs):
        segs=segs2
    seg=[]
    for i in range(len(segs)-1):
        seg.append(text[segs[i]:segs[i+1]])
    seg.append(text[segs[-1]:])
    return seg

In [4]:
seg_text(latex[1])

["-- 1. Express total work done in terms of Brenda and Brandon's rates:\n--    brendaRate = total_bricks / 9\n",
 '-- 2. Express total work done by Brandon:\n--    brandonRate = total_bricks / 10\n',
 '-- 3. Write the equation representing the combined work done by both Brenda and Brandon with a penalty applied:\n--    5 * ((brendaRate + brandonRate) - 10) = total_bricks\n',
 '-- 4. Solve for brendaRate + brandonRate:\n--    brendaRate + brandonRate = total_bricks / 9 + total_bricks / 10\n',
 '-- 5. Write down the expression for the penalty:\n--    Penalty = 10 (in bricks per hour)\n',
 '-- 6. Express 5 * ((brendaRate + brandonRate) - 10) in terms of total_bricks:\n--    5 * (total_bricks / 9 + total_bricks / 10 - 10) = total_bricks\n',
 '-- 7. Simplify the equation:\n--    Find common denominators and solve the equation 5 * ((total_bricks / 9) + (total_bricks / 10) - 10)\n',
 '-- 8. Conclude that total_bricks = 900:\n--    Verify that 5 * ((900 / 9) + (900 / 10) - 10) = 900 holds true

In [31]:
from openai import AzureOpenAI
client = AzureOpenAI(

)
deep_buffer={}
def get_proof(id,question,answer):
    if question in deep_buffer:
        return (id,deep_buffer[question])
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": '''
You are a math assistant. Here is the brief proof outline of the theorem.  
The problem is as follows:
%s
The outline of the proof is as follows:
%s
Please help me fill in the details between steps'''%(question,answer)
            }
        ]
    )
    result = response.choices[0].message.content
    deep_buffer[question]=result
    return (id,result)

In [30]:
#以下是多线程构造器
import concurrent.futures
def parallel_requests(func,prompts_list,max_process=16):
    results = [0]*len(prompts_list)
    # 使用线程池来并行处理请求
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_process) as executor:
        # 提交所有的请求到线程池
        futures = [executor.submit(func, id,item[0],item[1]) for id,item in enumerate(prompts_list)]
        # 收集所有请求的结果
        for future in concurrent.futures.as_completed(futures):
            try:
                id,result = future.result()
                results[id]= result
            except Exception as exc:
                results.append({"error": str(exc)})

    return results

In [38]:
# 构造数据对
prompts_list=[]
for i in range(len(datas)):
        prompts_list.append((datas[i]['informal_prefix'],latex[i]))
results = parallel_requests(get_proof,prompts_list[0:10])

In [44]:
results = parallel_requests(get_proof,prompts_list)

In [6]:
with open('1127con/requests.json','r') as f:
    results = json.load(f)

In [8]:
len(results) - len(datas)

0

# 构造half_shot
分成前半部分和后半部分，然后进行构造。

In [24]:
def con(a,b):
    ah = len(a)/2
    bh = len(b)/2
    con = a[int(ah):]+b[:int(bh)]
    return ''.join(con)

In [ ]:
def con2(a,b):
    results=[]
    for i in range(1,len(a)):
        for j in range(i):
            lena = int(j*len(a)/len(b))
            lenb = int(i*len(b)/len(a))
            con = a[:j]+[b[lena]]+a[j+1:i]+[b[lenb]]+a[i+1:]
            hint=''.join(con)
            results.append({'hint':hint,'id0':i,'id1':j})
    return results


In [11]:
new_con1=[]
for i in range(len(results)):
    new_con1.append((con(latex[i],results[i]),con(results[i],latex[i])))

In [12]:
new_con2=[]
z=0
for i in range(len(results)):
    try:
        new_con2.append(con2(seg_text(results[i]),seg_text(latex[i])))
        z+=1
    except:
        new_con2.append(latex[i])
print(z/len(results))

0.7695560253699789


In [13]:
new_con2[0]

[{'hint': '-- 1. Expand \\((x - y)^2\\) and \\((y - x)^2\\)\n-- 2. Show that \\((x - y)^2 = (y - x)^2\\)\n\n3. **Conclude that \\((x - y)^2 - (y - x)^2 = 0\\):**\n\n   Since \\((x - y)^2 = (y - x)^2\\), we have:\n   \\[\n   (x - y)^2 - (y - x)^2 = x^2 - 2xy + y^2 - (y^2 - 2xy + x^2) = 0.\n   \\]\n\n4. **Show that \\(0^2 = 0\\):**\n\n   The operation defined is \\(a$$$b = (a - b)^2\\). Applying this to \\(a = (x - y)^2\\) and \\(b = (y - x)^2\\), we have:\n   \\[\n   (x - y)^2$$$(y - x)^2 = ((x - y)^2 - (y - x)^2)^2 = 0^2 = 0.\n   \\]\n\nThus, the value of \\((x - y)^2$$$(y - x)^2\\) is \\(\\boxed{0}\\).',
  'id0': 1,
  'id1': 0},
 {'hint': '-- 1. Expand \\((x - y)^2\\) and \\((y - x)^2\\)\n\n2. **Show that \\((x - y)^2 = (y - x)^2\\):**\n\n   Both expressions are equal because both expand to \\(x^2 - 2xy + y^2\\).\n-- 3. Conclude that \\((x - y)^2 - (y - x)^2 = 0\\)\n\n4. **Show that \\(0^2 = 0\\):**\n\n   The operation defined is \\(a$$$b = (a - b)^2\\). Applying this to \\(a = (x - y

In [32]:
# with open('1127con/requests.json','w') as f:
#     json.dump(results,f)
with open('1127con/requests.json','r') as f:
    results=json.load(f)

seg_map = {}
for i in range(len(results)):
    name = datas[i]['name']
    try:
        seg_map[name] = seg_text(results[i])
    except:
        seg_map[name] = results[i]

In [34]:
with open('1127con/seg_map.json','w') as f:
    json.dump(seg_map,f)

# 回写第一个二分分割，首先需要加上一个自动注释器

In [14]:
def auto_comment(text):
    lines = text.splitlines()
    new_lines=[]
    for line in lines:
        if line.startswith('--'):
            new_lines.append(line)
        elif line.strip()=='':
            continue
        else:
            new_lines.append('-- '+line)
    return '\n'.join(new_lines)

In [15]:
new_ds_1=[]
for i in range(len(datas)):
    new_obj=datas[i].copy()
    new_obj2=datas[i].copy()
    new_obj['formal_statement']+='\n%s\n'%auto_comment(new_con1[i][0])
    new_obj2['formal_statement']+='\n%s\n'%auto_comment(new_con1[i][1])
    new_obj['kname']=new_obj['name']+'F'
    new_obj2['kname']=new_obj['name']+'B'
    new_ds_1.append(new_obj)
    new_ds_1.append(new_obj2)

In [28]:
new_ds_2=[]
for i in range(len(datas)):
    if type(new_con2[i])==str:
        new_obj=datas[i].copy()
        new_obj['formal_statement']+='\n%s\n'%auto_comment(new_con2[i])
        new_obj['kname']=new_obj['name']
        new_ds_2.append(new_obj)
        continue
    for j in range(len(new_con2[i])):
        new_obj=datas[i].copy()
        new_obj['formal_statement']+='\n%s\n'%auto_comment(new_con2[i][j]['hint'])
        new_obj['kname']=new_obj['name']+'_%d_%d'%(new_con2[i][j]['id1'],new_con2[i][j]['id0'])
        new_obj['src_hints']=seg_text(results[i])
        new_ds_2.append(new_obj)

In [17]:
new_obj

{'name': '2000_AIME_I_10',
 'split': 'test',
 'informal_prefix': 'A sequence of numbers $x_{1},x_{2},x_{3},\\ldots,x_{100}$ has the property that, for every integer $k$ between $1$ and $100,$ inclusive, the number $x_{k}$ is $k$ less than the sum of the other $99$ numbers. Given that $x_{50} = \\frac{m}{n}$, where $m$ and $n$ are relatively prime positive integers, find $m + n$.',
 'formal_statement': 'import Mathlib\nopen BigOperators Real Nat Topology Rat\ntheorem aime_problem_934 (m n : ℕ) (x : Fin 100 → ℚ)\n  (hmn_pos : m > 0 ∧ n > 0)\n  (h_coprime : Nat.Coprime m n)\n  (h_seq : ∀ k : Fin 100, \n    x k = (∑ i : Fin 100, x i) - x k - k)\n  (h_x50 : x ⟨49, ⟩ = m / n) :\n  m + n = 173 := by\n  \n-- 1. **Prove the Expression \\(\\sum_{i=1}^{100} x_i = 2x_k + k\\) for All \\( k \\):**\n--    The problem states: for each \\( k \\), \n--    \\[\n--    x_k = \\sum_{i=1, i \\neq k}^{100} x_i - k\n--    \\]\n--    which can be rewritten as:\n--    \\[\n--    \\sum_{i=1, i \\neq k}^{100} x_i

In [18]:
seg_text(results[i])

['\n1. **Prove the Expression \\(\\sum_{i=1}^{100} x_i = 2x_k + k\\) for All \\( k \\):**\n\n   The problem states: for each \\( k \\), \n\n   \\[\n   x_k = \\sum_{i=1, i \\neq k}^{100} x_i - k\n   \\]\n\n   which can be rewritten as:\n\n   \\[\n   \\sum_{i=1, i \\neq k}^{100} x_i = x_k + k\n   \\]\n\n   Thus, the total sum of all the numbers is:\n\n   \\[\n   \\sum_{i=1}^{100} x_i = x_k + \\sum_{i=1, i \\neq k}^{100} x_i = x_k + (x_k + k) = 2x_k + k\n   \\]\n\n   Hence, this equation holds for all \\( k \\).\n',
 '\n2. **Use \\( x_{50} = \\frac{m}{n} \\) to Find an Expression for \\(\\sum_{i=1}^{100} x_i\\):**\n\n   By setting \\( k = 50 \\) in the formula from step 1:\n\n   \\[\n   \\sum_{i=1}^{100} x_i = 2x_{50} + 50 = 2\\left(\\frac{m}{n}\\right) + 50 = \\frac{2m}{n} + 50\n   \\]\n\n   Therefore, the entire sum: \n\n   \\[\n   S = \\frac{2m}{n} + 50\n   \\]\n',
 '\n3. **Establish a Relation Between \\(\\sum_{i=1}^{100} x_i\\) and \\(m, n\\):**\n\n   Similarly, apply the formula for

In [19]:
#根据skip_list 进行过滤
with open('1127con/skip_list.json','r') as f:
    skip_list=json.load(f)

In [20]:
new1=[]
for item in new_ds_1:
    if item['name'] not in skip_list:
        temp = item.copy()
        temp ['name'] = temp['kname']
        new1.append(temp)

In [29]:
new2=[]
for item in new_ds_2:
    if item['name'] not in skip_list:
        temp = item.copy()
        temp ['name'] = temp['kname']
        new2.append(temp)

In [22]:
len(new2)

6868

In [30]:
#保存数据
with open('1127con/aime_amc_new_con1.jsonl','w') as f:
    for item in new1:
        f.write(json.dumps(item)+'\n')
with open('1127con/aime_amc_new_con2.jsonl','w') as f:
    for item in new2:
        f.write(json.dumps(item)+'\n')